In [144]:
import numpy as np 
import pandas as pd 
from datetime import date
#Data Source (Pandas Data Reader)
from pandas_datareader import data
import requests 
import json
from dateutil.relativedelta import relativedelta


#Data Source(Pandas Data Reader)


In [145]:
#entrylist =[['ANKR/USDT',0.07684],['AVAX/USDT',57],['WAXP/USDT',0.3818],['REEF/USDT',0.01346],['LINK/USDT',14.15],['AR/USDT',30],['ATOM/USDT',23],['MIR/USDT',2.343],['ALGO/USDT',0.909],['ZRX/USDT',0.90],['XTZ/USDT',2.856],['KSM/USDT',163],]
#entrylist =[['ANKRUSDT',0.07684],['AVAXUSDT',57],['WAXPUSDT',0.3818],['REEFUSDT',0.01131],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.60],['XTZUSDT',2.856],['KSMUSDT',163],['NEARUSDT',14],['WINUSDT',0.0003138],['CELRUSDT',0.05063],['DENTUSDT',0.002459],['LRCUSDT',0.7047],['CRVUSDT',3.529],['IMXUSDT',2.709],['SCRTUSDT',5],['RENUSDT',0.2547],['SUSHIUSDT',4.522],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.024],['MIRUSDT',2.343],['FLUXUSDT',1.523]]
entrylist =[['ANKRUSDT',0.07684],['AVAXUSDT',57],['WAXPUSDT',0.3174],['REEFUSDT',0.00930],['LINKUSDT',14.15],['ARUSDT',30],['ATOMUSDT',23],['MIRUSDT',2.343],['ALGOUSDT',0.909],['ZRXUSDT',0.60],['XTZUSDT',2.856],['KSMUSDT',150],['NEARUSDT',10.320],['WINUSDT',0.0003138],['DENTUSDT',0.002459],['LRCUSDT',0.7047],['CRVUSDT',3.529],['IMXUSDT',2.509],['SCRTUSDT',5],['RENUSDT',0.3047],['SUSHIUSDT',4],['CAKEUSDT',11.24],['XLMUSDT',0.2109],['HNTUSDT',30],['OMGUSDT',4.5],['LINAUSDT',0.024],['FLUXUSDT',1.523],['SRMUSDT',2.322],['VTHOUSDT',0.003938],['ANTUSDT',5.616],['RAYUSDT',3.54],['ROSEUSDT',0.25055],['MOVRUSDT',92.9],['TFUELUSDT',0.22],['GLMRUSDT',4.5],['COTIUSDT',0.2277],['ONEUSDT',0.13171],['VETUSDT',0.05138],['RUNEUSDT',3.709],['CELRUSDT',0.05063],['ONTUSDT',0.4652],['GALAUSDT',0.24794],['GRTUSDT',0.39],['DOTUSDT',16.31]]

startdate = '2020-01-01'
enddate = '2022-02-16'
interval = '4h'
easyband = pd.read_csv('easyband.csv')
#Columns=['Indicator','ALLTIME','CLOSERATIO','SMA100','SMA55']
easyband = pd.DataFrame(easyband,columns=['Indicator','ALLTIME','CLOSERATIO','SMA21','SMA55','GREENEASY'])
easyband = easyband.loc[:,['Indicator','SMA21','GREENEASY']]

#EASY=float(easyband.loc[(easyband.Indicator=='BANDUSDT')].SMA100.values)
#print(easyband)
#EASY[0]
#easyband

In [146]:
entries = pd.DataFrame(entrylist, columns =['Indicator', 'lowest_entry'],dtype = float) 
entries
entries.lowest_entry.astype(float)
indicator_list = entries['Indicator']
#indicator_list
#entries
entries.to_csv("entries.csv")

In [147]:
#whitelist=whitelist.lstrip()
#whitelist = indicator_list
#array = whitelist.split(",")
array = indicator_list
wl = []
bl = []
blacklist = ""
whitelist = ""
symbols =[]
for i in range(0 ,len(array)):
    if (array[i].find('DOWN') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])
    elif (array[i].find('UP') != -1):
        bl.append(str(array[i]).lstrip())
        #print ("Contains given substring ")
        blacklist += str(array[i])

    else:
        wl.append(str(array[i]).lstrip().replace("/",""))
        whitelist +=str(array[i]).lstrip()
        #print ("Doesn't contains given substring")
    

    
#symbols to be used to get the prices from binance url
symbols = np.array(wl)


## get the data 
import os

live = pd.to_datetime('today')
start = pd.to_datetime(startdate)
end = pd.to_datetime(enddate)

root_url = 'https://api.binance.com/api/v1/klines'
symbol = 'STEEMETH'
#interval = '1d'
url = root_url + '?symbol=' + symbol + '&interval=' + interval
print(url)


directory = 'BN-main'
table = []
   
        
        

for symbol in symbols:
        url = root_url + '?symbol=' + symbol + '&interval=' + interval
        data = json.loads(requests.get(url).text)
        dflive = pd.DataFrame(data ,columns =['open_time',
              'o', 'h', 'l', 'c', 'v',
              'close_time', 'qav', 'num_trades',
              'taker_base_vol', 'taker_quote_vol', 'ignore'])
        dflive.open_time = pd.to_datetime(dflive['open_time'],unit='ms')
        dflive=dflive.loc[(dflive.open_time>start)]
        lastrow = dflive.iloc[-1]
        now= float(lastrow.c)
        alth = dflive.c.max()
        altl = dflive.l.min()
        
        ## past 1 month 
        past_month = end - relativedelta(months=1)
        
        back1Mdata = dflive.loc[(dflive.open_time> past_month)]
        omh = back1Mdata.h.max()
        oml = back1Mdata.l.min()
        ## past 2 month
        
        past_2month = end - relativedelta(months=2)
        back2Mdata = dflive.loc[(dflive.open_time > past_2month)&(dflive.open_time < past_month)]
        tmh = back2Mdata.h.max()
        tml = back2Mdata.l.min()
        ## past 3 month 
        
        past_3month = end - relativedelta(months=3)
        back3Mdata = dflive.loc[(dflive.open_time> past_3month)&(dflive.open_time < past_2month)]
        thmh = back3Mdata.h.max()
        thml = back3Mdata.l.min()
        
        ## lowest entiry
        lowest = float(entries.loc[(entries.Indicator==symbol)].lowest_entry.values)
        #print(lowest)
        easyseries = easyband.loc[(easyband.Indicator==symbol)].GREENEASY.values
        #print(easyseries)
        if easyseries.size>0:
                green = easyseries[0]
        else:
                green = 0
        oeasyseries = easyband.loc[(easyband.Indicator==symbol)].SMA21.values
        #print(easyseries)
        if oeasyseries.size>0:
                orange = oeasyseries[0]
        else:
                orange = 0
 
        tuple =(symbol,alth,altl,now,omh,oml,tmh,tml,thmh,thml,lowest,green,orange)
        table.append(tuple)
        #print("name is: ",symbol ,alth,altl)
        

table[0]
AllTime = pd.DataFrame(table,columns=['Indicator','ATH','ATL','Current','OMH','OML','TMH','TML','THMH','THML','LOWEST','GREEN','ORANGE'])
AllTime.ATH=AllTime.ATH.astype(float)
AllTime.ATL=AllTime.ATL.astype(float)
AllTime.Current=AllTime.Current.astype(float)
AllTime.OMH=AllTime.OMH.astype(float)
AllTime.OML=AllTime.OML.astype(float)
AllTime.TMH=AllTime.TMH.astype(float)
AllTime.TML=AllTime.TML.astype(float)
AllTime.THMH=AllTime.THMH.astype(float)
AllTime.THML=AllTime.THML.astype(float)
AllTime.LOWEST=AllTime.LOWEST.astype(float)
AllTime.GREEN=AllTime.GREEN.astype(float)
AllTime.ORANGE=AllTime.ORANGE.astype(float)




AllTime.dtypes

AllTime['ALLTIME']=(AllTime['Current']/AllTime['ATH'])*100
AllTime['MONTH']=(AllTime['Current']/AllTime['OMH'])*100
AllTime['2MONTH']=(AllTime['Current']/AllTime['TMH'])*100
AllTime['3MONTH']=(AllTime['Current']/AllTime['THMH'])*100
AllTime['DROP']=((AllTime['Current']-AllTime['LOWEST'])/AllTime['LOWEST'])*100

AllTime.sort_values(by=['ALLTIME'],ascending=True,inplace = True)
#AllTime.sort_values(by='ATH',ascending=False)
#AllTime.to_csv("alltimeHL.csv", mode="w")
#data.sort_values("Name", axis = 0, ascending = True,inplace = True, na_position ='last')

## Display the Data 
AllTime.shape



https://api.binance.com/api/v1/klines?symbol=STEEMETH&interval=4h


(44, 18)

In [148]:
## Display the Data coins run last  Month

#drop = AllTime.loc[(AllTime['MONTH'] < 35)]
#drop = AllTime.loc[(AllTime['MONTH'] < 70)]
#drop = AllTime.loc[(AllTime['MONTH'] < 80)&(AllTime['2MONTH'] < 70)&(AllTime['ALLTIME'] < 40)]
drop = AllTime.loc[(AllTime['Current'] < AllTime['LOWEST'])]

#sort = drop.sort_values(by=['EASY'],ascending=False,inplace = True)
sort = drop.sort_values(by=['DROP'],ascending=True,inplace = True)

#drop = drop.loc[(drop['ALLTIME'] < drop['3MONTH'])]
display = drop[['Indicator','ORANGE','GREEN','DROP','LOWEST','Current']]

#print(display.iloc[0:60,0:14])
#print(display.iloc[60:110,])
#print(display.iloc[110:160,])
print("\b Fix Lowest entry ")

display

display.to_csv("fixentries.csv")

display

 Fix Lowest entry 


/Users/hjabbour/opt/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Indicator,ORANGE,GREEN,DROP,LOWEST,Current
7,MIRUSDT,2.250095,1.125048,-51.813914,2.343000,1.129000
34,GLMRUSDT,0.000000,0.000000,-49.044444,4.500000,2.293000
21,CAKEUSDT,12.943810,6.471905,-39.768683,11.240000,6.770000
17,IMXUSDT,0.000000,0.000000,-35.910721,2.509000,1.608000
16,CRVUSDT,0.000000,0.000000,-34.995750,3.529000,2.294000
33,TFUELUSDT,0.000000,0.000000,-30.000000,0.220000,0.154000
23,HNTUSDT,0.000000,0.000000,-28.100000,30.000000,21.570000
39,CELRUSDT,0.088916,0.044458,-27.592337,0.050630,0.036660
32,MOVRUSDT,0.000000,0.000000,-27.125942,92.900000,67.700000
30,RAYUSDT,7.329190,3.664595,-25.084746,3.540000,2.652000


In [149]:
display = display.loc[display.GREEN > 0]


In [150]:
## Green entry
print("\b Enries on easy band")
display

 Enries on easy band


,Indicator,ORANGE,GREEN,DROP,LOWEST,Current
7,MIRUSDT,2.250095,1.125048,-51.813914,2.343000,1.129000
21,CAKEUSDT,12.943810,6.471905,-39.768683,11.240000,6.770000
39,CELRUSDT,0.088916,0.044458,-27.592337,0.050630,0.036660
30,RAYUSDT,7.329190,3.664595,-25.084746,3.540000,2.652000
25,LINAUSDT,0.042515,0.021257,-21.833333,0.024000,0.018760
28,VTHOUSDT,0.006722,0.003361,-21.432199,0.003938,0.003094
20,SUSHIUSDT,7.334476,3.667238,-19.125000,4.000000,3.235000
27,SRMUSDT,4.478619,2.239310,-19.121447,2.322000,1.878000
5,ARUSDT,49.643810,24.821905,-18.933333,30.000000,24.320000
11,KSMUSDT,290.919048,145.459524,-16.000000,150.000000,126.000000


In [151]:
display = display.loc[display['Current'] < display['GREEN']]
display

,Indicator,ORANGE,GREEN,DROP,LOWEST,Current
39,CELRUSDT,0.088916,0.044458,-27.592337,0.050630,0.036660
30,RAYUSDT,7.329190,3.664595,-25.084746,3.540000,2.652000
25,LINAUSDT,0.042515,0.021257,-21.833333,0.024000,0.018760
28,VTHOUSDT,0.006722,0.003361,-21.432199,0.003938,0.003094
20,SUSHIUSDT,7.334476,3.667238,-19.125000,4.000000,3.235000
27,SRMUSDT,4.478619,2.239310,-19.121447,2.322000,1.878000
5,ARUSDT,49.643810,24.821905,-18.933333,30.000000,24.320000
11,KSMUSDT,290.919048,145.459524,-16.000000,150.000000,126.000000
24,OMGUSDT,8.135810,4.067905,-15.933333,4.500000,3.783000
37,VETUSDT,0.094038,0.047019,-12.358895,0.051380,0.045030
